In [ ]:
import psycopg2 as pg
import panel as pn

# Inicializar o Panel
pn.extension()

# Função para conectar ao banco
def get_db_connection():
    return pg.connect(host='localhost', dbname='SocioEcoData', user='postgres', password='1234')

#  Campos do Formulário de Cadastro
nome = pn.widgets.TextInput(name="Nome", placeholder="Digite o nome")
email = pn.widgets.TextInput(name="Email", placeholder="Digite o email")
senha = pn.widgets.PasswordInput(name="Senha", placeholder="Digite a senha")
idioma = pn.widgets.Select(
    name='Idioma Preferido',
    options=['Português', 'Inglês', 'Espanhol', 'Francês']
)

# Painel de saída para mensagens
output_message = pn.pane.Alert("", alert_type="success", visible=False)

# Função para Salvar Usuário
def salvar_usuario(event):
    global output_message

    if not nome.value.strip() or not email.value.strip() or not senha.value.strip():
        output_message.object = "⚠️ Preencha todos os campos antes de cadastrar!"
        output_message.alert_type = "warning"
        output_message.visible = True
        return  
    
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute(
            "INSERT INTO Perfil (nome, email, senha) VALUES (%s, %s, %s) RETURNING id_perfil;",
            (nome.value, email.value, senha.value)
        )
        id_perfil = cur.fetchone()[0]  

        cur.execute(
            "INSERT INTO Usuario (id_perfil, idioma_preferido) VALUES (%s, %s);",
            (id_perfil, idioma.value)
        )

        con.commit()

        output_message.object = "✅ Usuário cadastrado com sucesso!"
        output_message.alert_type = "success"
        output_message.visible = True

        nome.value = ""
        email.value = ""
        senha.value = ""
        idioma.value = "Português"

    except Exception as e:
        con.rollback()  
        output_message.object = f"❌ Erro ao cadastrar usuário: {e}"
        output_message.alert_type = "danger"
        output_message.visible = True

    finally:
        cur.close()
        con.close()

# Botões
botao_cadastrar = pn.widgets.Button(name="Cadastrar", button_type="primary")
botao_cadastrar.on_click(salvar_usuario)

botao_voltar = pn.widgets.Button(name="🔙 Voltar", button_type="warning")

# Tela de Cadastro
tela_cadastro = pn.Column(
    "# 📝 Cadastro de Usuário",
    nome, email, senha, idioma, 
    botao_cadastrar,
    output_message,
    botao_voltar  
)


# Tela de Login
login_email = pn.widgets.TextInput(name="Email", placeholder="Digite seu email")
login_senha = pn.widgets.PasswordInput(name="Senha", placeholder="Digite sua senha")

# Novo Select para escolher entre Usuário e Administrador
login_tipo = pn.widgets.Select(
    name="Tipo de Conta",
    options=["Usuário", "Administrador"]
)

login_output = pn.pane.Alert("", alert_type="danger", visible=False)

botao_entrar = pn.widgets.Button(name="Entrar", button_type="primary")
botao_voltar_login = pn.widgets.Button(name="🔙 Voltar", button_type="warning")

# Criando a Tela de Login
tela_login = pn.Column(
    "# 🔐 Login",
    login_email,
    login_senha,
    login_tipo,  
    botao_entrar,
    login_output,
    botao_voltar_login  
)

# Tela do Administrador
admin_email_busca = pn.widgets.TextInput(name="Buscar Usuário por Email", placeholder="Digite o email")
admin_resultado = pn.pane.Alert("", alert_type="info", visible=False)
botao_buscar = pn.widgets.Button(name="Buscar Usuário", button_type="primary")
botao_voltar_admin = pn.widgets.Button(name="🔙 Voltar", button_type="warning")
botao_voltar_admin_cruds = pn.widgets.Button(name="🔙 Voltar", button_type="warning")

# Campos de Edição (inicialmente ocultos)
admin_nome_editar = pn.widgets.TextInput(name="Nome", placeholder="Novo nome", visible=False)
admin_email_editar = pn.widgets.TextInput(name="Email", placeholder="Novo email", visible=False)
admin_senha_editar = pn.widgets.PasswordInput(name="Senha", placeholder="Nova senha", visible=False)
botao_atualizar = pn.widgets.Button(name="Salvar Alterações", button_type="success", visible=False)
admin_atualizar_output = pn.pane.Alert("", alert_type="success", visible=False)


# Botões para CRUD de Índices Econômicos
botao_adicionar_indice = pn.widgets.Button(name="➕ Adicionar Índice", button_type="success")
botao_editar_indice = pn.widgets.Button(name="✏️ Editar Índice", button_type="warning")
botao_excluir_indice = pn.widgets.Button(name="🗑️ Excluir Índice", button_type="danger")
botao_consultar_indices = pn.widgets.Button(name="🔍 Consultar Índices", button_type="primary")

# Widgets da Tela de Alteração de Valor do Índice
valor_indice_lista = pn.widgets.Select(name="Escolha um Índice Econômico", options=[])
botao_carregar_valor_indice = pn.widgets.Button(name="Carregar Valores", button_type="primary")

valor_data = pn.widgets.DatePicker(name="Data de Referência")
valor_numero = pn.widgets.TextInput(name="Valor", placeholder="Digite o valor do índice")

botao_salvar_valor = pn.widgets.Button(name="Adicionar Valor", button_type="success")
botao_atualizar_valor = pn.widgets.Button(name="Atualizar Valor", button_type="warning")
botao_excluir_valor = pn.widgets.Button(name="Excluir Valor", button_type="danger")

valor_indice_output = pn.pane.Alert("", alert_type="success", visible=False)

# Variável de controle para verificar se o usuário foi encontrado
usuario_encontrado = False

# Toggle de edição (inicialmente desativado)
mostrar_campos_edicao = pn.widgets.Toggle(name="Modo Edição", button_type="primary", value=False, disabled=True)

# Função para buscar usuários no Painel do Administrador
def buscar_usuario(event):
    global admin_resultado, usuario_encontrado
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute(
            "SELECT nome, email FROM Perfil WHERE email = %s;",
            (admin_email_busca.value,)
        )
        
        resultado = cur.fetchone()

        if resultado:
            admin_resultado.object = f"✅ Nome: {resultado[0]} | Email: {resultado[1]}"
            admin_resultado.alert_type = "success"

            # Preencher os campos editáveis com os valores atuais
            admin_nome_editar.value = resultado[0]
            admin_email_editar.value = resultado[1]
            admin_senha_editar.value = ""

            botao_atualizar.disabled = False  # Habilitar botão de atualização
            mostrar_campos_edicao.disabled = False  # Habilitar botão "Modo Edição"
            usuario_encontrado = True  # Marcar que um usuário foi encontrado

            botao_excluir.visible = True  # Tornar o botão de exclusão visível
        else:
            admin_resultado.object = "❌ Usuário não encontrado!"
            admin_resultado.alert_type = "danger"
            botao_atualizar.disabled = True  # Desabilitar atualização
            mostrar_campos_edicao.disabled = True  # Desabilitar edição
            usuario_encontrado = False  # Marcar que nenhum usuário foi encontrado
            botao_excluir.visible = False  # Ocultar o botão de exclusão caso não encontre


        admin_resultado.visible = True

    except Exception as e:
        admin_resultado.object = f"❌ Erro ao buscar usuário: {e}"
        admin_resultado.alert_type = "danger"
        admin_resultado.visible = True
        mostrar_campos_edicao.disabled = True  # Garantir que o botão fique desabilitado
        usuario_encontrado = False

    finally:
        cur.close()
        con.close()


def atualizar_usuario(event):
    global admin_atualizar_output

    # Verificação dos campos vazios
    if not admin_nome_editar.value.strip() or not admin_email_editar.value.strip() or not admin_senha_editar.value.strip():
        admin_atualizar_output.object = "⚠️ Todos os campos devem estar preenchidos antes de atualizar!"
        admin_atualizar_output.alert_type = "warning"
        admin_atualizar_output.visible = True
        return  # Sai da função sem tentar atualizar

    try:
        con = get_db_connection()
        cur = con.cursor()

        # Atualizar os dados na tabela Perfil
        cur.execute("""
            UPDATE Perfil 
            SET nome = %s, email = %s, senha = %s
            WHERE email = %s;
        """, (admin_nome_editar.value, admin_email_editar.value, admin_senha_editar.value, admin_email_busca.value))

        con.commit()

        admin_atualizar_output.object = "✅ Dados do usuário atualizados com sucesso!"
        admin_atualizar_output.alert_type = "success"
        admin_atualizar_output.visible = True

        # Atualizar o campo de email buscado para refletir a alteração
        admin_email_busca.value = admin_email_editar.value

    except Exception as e:
        con.rollback()
        admin_atualizar_output.object = f"❌ Erro ao atualizar usuário: {e}"
        admin_atualizar_output.alert_type = "danger"
        admin_atualizar_output.visible = True

    finally:
        cur.close()
        con.close()


def alternar_modo_edicao(event):
    if not usuario_encontrado:  # Se nenhum usuário foi encontrado, impede a exibição
        mostrar_campos_edicao.value = False
        return
    
    estado = mostrar_campos_edicao.value
    admin_nome_editar.visible = estado
    admin_email_editar.visible = estado
    admin_senha_editar.visible = estado
    botao_atualizar.visible = estado


botao_excluir = pn.widgets.Button(name="🗑️ Excluir Usuário", button_type="danger", visible=False)

def excluir_usuario(event):
    global admin_atualizar_output, usuario_encontrado  # Agora a variável é reconhecida corretamente


    # Verificar se há um usuário buscado antes de excluir
    if not usuario_encontrado:
        admin_atualizar_output.object = "⚠️ Nenhum usuário foi buscado para exclusão!"
        admin_atualizar_output.alert_type = "warning"
        admin_atualizar_output.visible = True
        return

    try:
        con = get_db_connection()
        cur = con.cursor()

        # Excluir o usuário da tabela Perfil (cascateando para as tabelas dependentes)
        cur.execute("DELETE FROM Perfil WHERE email = %s;", (admin_email_busca.value,))
        con.commit()

        admin_atualizar_output.object = "✅ Usuário excluído com sucesso!"
        admin_atualizar_output.alert_type = "success"
        admin_atualizar_output.visible = True

        # Resetar campos após exclusão
        admin_email_busca.value = ""
        admin_nome_editar.value = ""
        admin_email_editar.value = ""
        admin_senha_editar.value = ""

        # Desabilitar botões e edição após exclusão
        botao_atualizar.visible = False
        botao_excluir.visible = False
        mostrar_campos_edicao.disabled = True
        mostrar_campos_edicao.value = False  # Resetar a edição
        usuario_encontrado = False  # Resetar a variável de controle

    except Exception as e:
        con.rollback()
        admin_atualizar_output.object = f"❌ Erro ao excluir usuário: {e}"
        admin_atualizar_output.alert_type = "danger"
        admin_atualizar_output.visible = True

    finally:
        cur.close()
        con.close()


botao_excluir.on_click(excluir_usuario)

# Conectar o botão "Editar" à função
mostrar_campos_edicao.param.watch(alternar_modo_edicao, 'value')

# Conectar botão de busca à função
botao_buscar.on_click(buscar_usuario)


# Botões para abrir as telas de Usuário e Índices
botao_alterar_usuario = pn.widgets.Button(name="👤 Alterar Usuário", button_type="primary")
botao_alterar_indices = pn.widgets.Button(name="📊 Alterar Índices", button_type="success")
botao_alterar_valor_indice = pn.widgets.Button(name="📈 Alterar Valor Índice", button_type="warning")


# Tela do Administrador
tela_admin = pn.Column(
    "# 👨‍💼 Painel do Administrador",
    botao_alterar_usuario,
    botao_alterar_indices,
    botao_alterar_valor_indice,
    botao_voltar_admin
)

# Tela para Gerenciar Usuários
tela_alterar_usuario = pn.Column(
    "# 👤 Gerenciar Usuários",
    admin_email_busca,
    botao_buscar,
    mostrar_campos_edicao,
    pn.pane.Markdown("### ✏️ Editar Usuário"),
    admin_nome_editar,
    admin_email_editar,
    admin_senha_editar,
    botao_atualizar,
    botao_excluir,
    admin_atualizar_output,
    botao_voltar_admin_cruds
)

# Tela para Gerenciar Índices
tela_alterar_indices = pn.Column(
    "# 📊 Gerenciar Índices Econômicos",
    botao_adicionar_indice,
    botao_editar_indice,
    botao_excluir_indice,
    botao_consultar_indices,
    botao_voltar_admin_cruds
)

# Tela de Alterar Valor do Índice
tela_alterar_valor_indice = pn.Column(
    "# 📈 Alterar Valor de Índices Econômicos",
    valor_indice_lista,
    botao_carregar_valor_indice,
    valor_data,
    valor_numero,
    botao_salvar_valor,
    botao_atualizar_valor,
    botao_excluir_valor,
    valor_indice_output,
    botao_voltar_admin_cruds
)

# Campos para Gerenciamento de Índices
indice_nome = pn.widgets.TextInput(name="Nome do Índice", placeholder="Digite o nome do índice")
indice_descricao = pn.widgets.TextAreaInput(name="Descrição", placeholder="Descreva o índice")
indice_unidade = pn.widgets.TextInput(name="Unidade", placeholder="Exemplo: %, R$, pontos")
botao_salvar_indice = pn.widgets.Button(name="Salvar Índice", button_type="success")
indice_output = pn.pane.Alert("", alert_type="success", visible=False)
botao_voltar_indices = pn.widgets.Button(name="🔙 Voltar", button_type="warning")

# Tela para Adicionar Índice
tela_adicionar_indice = pn.Column(
    "# ➕ Adicionar Novo Índice Econômico",
    indice_nome,
    indice_descricao,
    indice_unidade,
    botao_salvar_indice,
    indice_output,
    botao_voltar_indices
)

# Tela para Editar Índice
indice_lista = pn.widgets.Select(name="Escolha um Índice", options=[])
botao_carregar_indice = pn.widgets.Button(name="Carregar Índice", button_type="primary")
indice_editar_nome = pn.widgets.TextInput(name="Nome do Índice", placeholder="Novo nome")
indice_editar_descricao = pn.widgets.TextAreaInput(name="Descrição", placeholder="Nova descrição")
indice_editar_unidade = pn.widgets.TextInput(name="Unidade", placeholder="Nova unidade")
botao_atualizar_indice = pn.widgets.Button(name="Atualizar Índice", button_type="success")
indice_editar_output = pn.pane.Alert("", alert_type="success", visible=False)

tela_editar_indice = pn.Column(
    "# ✏️ Editar Índice Econômico",
    indice_lista,
    botao_carregar_indice,
    indice_editar_nome,
    indice_editar_descricao,
    indice_editar_unidade,
    botao_atualizar_indice,
    indice_editar_output,
    botao_voltar_indices
)

# Tela para Excluir Índice
indice_excluir_lista = pn.widgets.Select(name="Escolha um Índice", options=[])
botao_excluir_indice_conf = pn.widgets.Button(name="Excluir Índice", button_type="danger")
indice_excluir_output = pn.pane.Alert("", alert_type="danger", visible=False)

tela_excluir_indice = pn.Column(
    "# 🗑️ Excluir Índice Econômico",
    indice_excluir_lista,
    botao_excluir_indice_conf,
    indice_excluir_output,
    botao_voltar_indices
)

# Tela de Consulta de Índices
indice_consulta_lista = pn.widgets.Select(name="Índice Econômico", options=[])
botao_consultar_indice = pn.widgets.Button(name="Consultar Índice", button_type="primary")
resultado_consulta_indice = pn.pane.Alert("", alert_type="info", visible=False)

tela_consulta_indice = pn.Column(
    "# 🔍 Consultar Índices Econômicos",
    indice_consulta_lista,
    botao_consultar_indice,
    resultado_consulta_indice,
    botao_voltar_indices
)

# Tela de Consulta de Índices Econômicos
filtro_indice = pn.widgets.Select(name="Índice Econômico", options=[])
botao_consultar = pn.widgets.Button(name="Consultar Índice", button_type="primary")
resultado_consulta = pn.pane.Alert("", alert_type="info", visible=False)
botao_voltar_consulta = pn.widgets.Button(name="🔙 Voltar", button_type="warning")

# Função para buscar os índices econômicos disponíveis
def carregar_indices():
    try:
        con = get_db_connection()
        cur = con.cursor()
        cur.execute("SELECT nome FROM Indice_Economico;")
        indices = [row[0] for row in cur.fetchall()]
        filtro_indice.options = indices if indices else ["Nenhum índice disponível"]
    except Exception as e:
        filtro_indice.options = ["Erro ao carregar índices"]
    finally:
        cur.close()
        con.close()

# Função para consultar valores do índice selecionado
def consultar_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()
        cur.execute("""
            SELECT data_referencia, valor 
            FROM Valor_Indice vi
            JOIN Indice_Economico ie ON vi.id_indice = ie.id_indice
            WHERE ie.nome = %s
            ORDER BY data_referencia DESC LIMIT 5;
        """, (filtro_indice.value,))
        resultados = cur.fetchall()

        if resultados:
            mensagem = "\n".join([f"📅 {row[0]} → 💰 {row[1]}" for row in resultados])
            resultado_consulta.object = f"📊 Últimos valores de {filtro_indice.value}:\n\n{mensagem}"
            resultado_consulta.alert_type = "success"
        else:
            resultado_consulta.object = f"⚠️ Nenhum dado encontrado para {filtro_indice.value}"
            resultado_consulta.alert_type = "warning"

        resultado_consulta.visible = True

    except Exception as e:
        resultado_consulta.object = f"❌ Erro ao consultar índice: {e}"
        resultado_consulta.alert_type = "danger"
        resultado_consulta.visible = True
    finally:
        cur.close()
        con.close()

# Conectar botão à função de consulta
botao_consultar.on_click(consultar_indice)

# Criando a tela de consulta
tela_consulta = pn.Column(
    "# 📈 Consulta de Índices Econômicos",
    filtro_indice,
    botao_consultar,
    resultado_consulta,
    botao_voltar_consulta
)

# Função para Carregar Índices
def carregar_dados_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()
        
        cur.execute("SELECT descricao, unidade FROM Indice_Economico WHERE nome = %s;", (indice_lista.value,))
        resultado = cur.fetchone()
        
        if resultado:
            indice_editar_nome.value = indice_lista.value  # Mantém o nome original
            indice_editar_descricao.value = resultado[0]
            indice_editar_unidade.value = resultado[1]
    
    except Exception as e:
        indice_editar_output.object = f"❌ Erro ao carregar índice: {e}"
        indice_editar_output.alert_type = "danger"
        indice_editar_output.visible = True
    
    finally:
        cur.close()
        con.close()

# Conectar ao botão de carregar índice
botao_carregar_indice.on_click(carregar_dados_indice)

def carregar_lista_indices():
    try:
        con = get_db_connection()
        cur = con.cursor()
        cur.execute("SELECT nome FROM Indice_Economico;")
        indices = [row[0] for row in cur.fetchall()]
        
        # Atualiza a lista de opções para seleção nos formulários
        indice_lista.options = indices if indices else ["Nenhum índice disponível"]
        indice_excluir_lista.options = indices if indices else ["Nenhum índice disponível"]
        indice_consulta_lista.options = indices if indices else ["Nenhum índice disponível"]

        # Também atualiza a lista de índices para a tela de Alterar Valor Índice
        valor_indice_lista.options = indices if indices else ["Nenhum índice disponível"]
        
    except Exception as e:
        indice_lista.options = ["Erro ao carregar índices"]
        indice_excluir_lista.options = ["Erro ao carregar índices"]
        indice_consulta_lista.options = ["Erro ao carregar índices"]
        valor_indice_lista.options = ["Erro ao carregar índices"]  # Incluído para a nova funcionalidade
    finally:
        cur.close()
        con.close()


# Função para Adicionar Índice
def adicionar_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()
        cur.execute(
            "INSERT INTO Indice_Economico (nome, descricao, unidade) VALUES (%s, %s, %s);",
            (indice_nome.value, indice_descricao.value, indice_unidade.value)
        )
        con.commit()
        indice_output.object = "✅ Índice adicionado com sucesso!"
        indice_output.alert_type = "success"
        indice_output.visible = True
        carregar_lista_indices()
    except Exception as e:
        indice_output.object = f"❌ Erro ao adicionar índice: {e}"
        indice_output.alert_type = "danger"
        indice_output.visible = True
    finally:
        cur.close()
        con.close()

botao_salvar_indice.on_click(adicionar_indice)

# Função para Atualizar Índice
def atualizar_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()

        # Obtemos o ID do índice com base no nome selecionado
        cur.execute("SELECT id_indice FROM Indice_Economico WHERE nome = %s;", (indice_lista.value,))
        id_indice = cur.fetchone()

        if not id_indice:
            indice_editar_output.object = "⚠️ Índice não encontrado para atualização!"
            indice_editar_output.alert_type = "warning"
            indice_editar_output.visible = True
            return

        id_indice = id_indice[0]  # Extrai o ID real da tupla

        # Atualiza o índice usando `id_indice` como identificador único
        cur.execute(
            "UPDATE Indice_Economico SET nome = %s, descricao = %s, unidade = %s WHERE id_indice = %s;",
            (indice_editar_nome.value, indice_editar_descricao.value, indice_editar_unidade.value, id_indice)
        )

        con.commit()

        indice_editar_output.object = "✅ Índice atualizado com sucesso!"
        indice_editar_output.alert_type = "success"
        indice_editar_output.visible = True

        # Recarrega a lista de índices após a atualização
        carregar_lista_indices()

    except Exception as e:
        con.rollback()
        indice_editar_output.object = f"❌ Erro ao atualizar índice: {e}"
        indice_editar_output.alert_type = "danger"
        indice_editar_output.visible = True
    finally:
        cur.close()
        con.close()


# Conectar o botão à função de atualização
botao_atualizar_indice.on_click(atualizar_indice)


# Função para Excluir Índice
def excluir_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute("DELETE FROM Indice_Economico WHERE nome = %s;", (indice_excluir_lista.value,))
        con.commit()

        indice_excluir_output.object = "✅ Índice excluído com sucesso!"
        indice_excluir_output.alert_type = "success"
        indice_excluir_output.visible = True

        carregar_lista_indices()  # Atualiza a lista de índices após exclusão

    except Exception as e:
        indice_excluir_output.object = f"❌ Erro ao excluir índice: {e}"
        indice_excluir_output.alert_type = "danger"
        indice_excluir_output.visible = True

    finally:
        cur.close()
        con.close()

# Conectar o botão ao evento de exclusão
botao_excluir_indice_conf.on_click(excluir_indice)


# Função para Consultar Índices
def consultar_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute("SELECT descricao, unidade FROM Indice_Economico WHERE nome = %s;", (indice_consulta_lista.value,))
        resultado = cur.fetchone()

        if resultado:
            resultado_consulta_indice.object = f"📊 {indice_consulta_lista.value}\n\n📄 Descrição: {resultado[0]}\n Unidade: {resultado[1]}"
            resultado_consulta_indice.alert_type = "success"
        else:
            resultado_consulta_indice.object = "⚠️ Nenhum dado encontrado."
            resultado_consulta_indice.alert_type = "warning"

        resultado_consulta_indice.visible = True

    except Exception as e:
        resultado_consulta_indice.object = f"❌ Erro ao consultar índice: {e}"
        resultado_consulta_indice.alert_type = "danger"
        resultado_consulta_indice.visible = True

    finally:
        cur.close()
        con.close()

# Conectar botão à função de consulta
botao_consultar_indice.on_click(consultar_indice)

def carregar_valores_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()
        cur.execute("""
            SELECT data_referencia, valor FROM Valor_Indice vi
            JOIN Indice_Economico ie ON vi.id_indice = ie.id_indice
            WHERE ie.nome = %s ORDER BY data_referencia DESC;
        """, (valor_indice_lista.value,))
        resultado = cur.fetchone()

        if resultado:
            valor_data.value = resultado[0]
            valor_numero.value = str(resultado[1])  # Converte float para string
            valor_indice_output.object = "✅ Valores carregados com sucesso!"
            valor_indice_output.alert_type = "success"
        else:
            valor_data.value = None
            valor_numero.value = ""  # Ajuste: Define como string vazia se não houver valor
            valor_indice_output.object = "⚠️ Nenhum valor encontrado para este Índice Econômico."
            valor_indice_output.alert_type = "warning"

        valor_indice_output.visible = True

    except Exception as e:
        valor_indice_output.object = f"❌ Erro ao carregar valores: {e}"
        valor_indice_output.alert_type = "danger"
        valor_indice_output.visible = True
    finally:
        cur.close()
        con.close()




def adicionar_valor_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute("SELECT id_indice FROM Indice_Economico WHERE nome = %s;", (valor_indice_lista.value,))
        id_indice = cur.fetchone()

        if not id_indice:
            valor_indice_output.object = "⚠️ Índice não encontrado!"
            valor_indice_output.alert_type = "warning"
            valor_indice_output.visible = True
            return
        
        id_indice = id_indice[0]
        
        cur.execute("""
            INSERT INTO Valor_Indice (id_indice, data_referencia, valor)
            VALUES (%s, %s, %s);
        """, (id_indice, valor_data.value, valor_numero.value))

        con.commit()
        valor_indice_output.object = "✅ Valor adicionado com sucesso!"
        valor_indice_output.alert_type = "success"
        valor_indice_output.visible = True

    except Exception as e:
        con.rollback()
        valor_indice_output.object = f"❌ Erro ao adicionar valor: {e}"
        valor_indice_output.alert_type = "danger"
        valor_indice_output.visible = True
    finally:
        cur.close()
        con.close()

def atualizar_valor_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute("SELECT id_indice FROM Indice_Economico WHERE nome = %s;", (valor_indice_lista.value,))
        id_indice = cur.fetchone()

        if not id_indice:
            valor_indice_output.object = "⚠️ Índice não encontrado!"
            valor_indice_output.alert_type = "warning"
            valor_indice_output.visible = True
            return
        
        id_indice = id_indice[0]
        
        cur.execute("""
            UPDATE Valor_Indice
            SET valor = %s
            WHERE id_indice = %s AND data_referencia = %s;
        """, (valor_numero.value, id_indice, valor_data.value))

        con.commit()
        valor_indice_output.object = "✅ Valor atualizado com sucesso!"
        valor_indice_output.alert_type = "success"
        valor_indice_output.visible = True

    except Exception as e:
        con.rollback()
        valor_indice_output.object = f"❌ Erro ao atualizar valor: {e}"
        valor_indice_output.alert_type = "danger"
        valor_indice_output.visible = True
    finally:
        cur.close()
        con.close()

def excluir_valor_indice(event):
    try:
        con = get_db_connection()
        cur = con.cursor()

        cur.execute("SELECT id_indice FROM Indice_Economico WHERE nome = %s;", (valor_indice_lista.value,))
        id_indice = cur.fetchone()

        if not id_indice:
            valor_indice_output.object = "⚠️ Índice não encontrado!"
            valor_indice_output.alert_type = "warning"
            valor_indice_output.visible = True
            return
        
        id_indice = id_indice[0]
        
        cur.execute("""
            DELETE FROM Valor_Indice
            WHERE id_indice = %s AND data_referencia = %s;
        """, (id_indice, valor_data.value))

        con.commit()
        valor_indice_output.object = "✅ Valor excluído com sucesso!"
        valor_indice_output.alert_type = "success"
        valor_indice_output.visible = True

    except Exception as e:
        con.rollback()
        valor_indice_output.object = f"❌ Erro ao excluir valor: {e}"
        valor_indice_output.alert_type = "danger"
        valor_indice_output.visible = True
    finally:
        cur.close()
        con.close()

botao_carregar_valor_indice.on_click(carregar_valores_indice)
botao_salvar_valor.on_click(adicionar_valor_indice)
botao_atualizar_valor.on_click(atualizar_valor_indice)
botao_excluir_valor.on_click(excluir_valor_indice)



# Funções para abrir as novas telas
def abrir_tela_alterar_usuario(event):
    main_area.objects = [tela_alterar_usuario]

def abrir_tela_alterar_indices(event):
    main_area.objects = [tela_alterar_indices]

def abrir_tela_alterar_valor_indice(event):
    carregar_lista_indices()
    main_area.objects = [tela_alterar_valor_indice]


# Conectar os botões às funções
botao_alterar_usuario.on_click(abrir_tela_alterar_usuario)
botao_alterar_indices.on_click(abrir_tela_alterar_indices)
botao_alterar_valor_indice.on_click(abrir_tela_alterar_valor_indice)

# Abrir Tela de Adicionar Índice
def abrir_tela_adicionar_indice(event):
    main_area.objects = [tela_adicionar_indice]

# Abrir Tela de Editar Índice
def abrir_tela_editar_indice(event):
    carregar_lista_indices()  # Certifique-se de carregar os índices antes de exibir a tela
    main_area.objects = [tela_editar_indice]

# Abrir Tela de Excluir Índice
def abrir_tela_excluir_indice(event):
    carregar_lista_indices()  # Certifique-se de carregar os índices antes de exibir a tela
    main_area.objects = [tela_excluir_indice]

#  Abrir Tela de Consultar Índices
def abrir_tela_consultar_indices(event):
    carregar_lista_indices()  #  Certifique-se de carregar os índices antes de exibir a tela
    main_area.objects = [tela_consulta_indice]

# Conectar os botões às funções corretas
botao_adicionar_indice.on_click(abrir_tela_adicionar_indice)
botao_editar_indice.on_click(abrir_tela_editar_indice)
botao_excluir_indice.on_click(abrir_tela_excluir_indice)
botao_consultar_indices.on_click(abrir_tela_consultar_indices)

#  Função para validar login
def validar_login(event):
    global login_output
    try:
        con = get_db_connection()
        cur = con.cursor()

        if login_tipo.value == "Usuário":
            cur.execute("""
                SELECT COUNT(*) 
                FROM Perfil p
                JOIN Usuario u ON p.id_perfil = u.id_perfil
                WHERE p.email = %s AND p.senha = %s;
            """, (login_email.value, login_senha.value))
        else:
            cur.execute("""
                SELECT COUNT(*) 
                FROM Perfil p
                JOIN Administrador a ON p.id_perfil = a.id_perfil
                WHERE p.email = %s AND p.senha = %s;
            """, (login_email.value, login_senha.value))

        resultado = cur.fetchone()[0]  

        if resultado == 1:
            if login_tipo.value == "Administrador":
                main_area.objects = [tela_admin]  #  Se for administrador, abre o painel do admin
            else:
                carregar_indices()  # Carregar opções antes de mostrar a tela
                main_area.objects = [tela_consulta]  #  Se for usuário, abre a tela de consulta

        else:
            login_output.object = "❌ Email ou senha incorretos!"
            login_output.alert_type = "danger"
            login_output.visible = True

    except Exception as e:
        login_output.object = f"❌ Erro ao processar login: {e}"
        login_output.alert_type = "danger"
        login_output.visible = True

    finally:
        cur.close()
        con.close()


# Conectar botão de login à função de autenticação
botao_entrar.on_click(validar_login)

botao_atualizar.on_click(atualizar_usuario)


#  Função para voltar para a tela inicial
def voltar_para_tela_inicial(event):
    main_area.objects = [tela_inicial()]

#  Função para voltar para o Painel do Administrador
def voltar_para_admin(event):
    main_area.objects = [tela_admin]

def voltar_para_indices(event):
    main_area.objects = [tela_alterar_indices]



# Conectar botões "Voltar" às funções de retorno
botao_voltar_indices.on_click(voltar_para_indices)
botao_voltar.on_click(voltar_para_tela_inicial)
botao_voltar_login.on_click(voltar_para_tela_inicial)
botao_voltar_admin.on_click(voltar_para_tela_inicial)
botao_voltar_admin_cruds.on_click(voltar_para_admin)
botao_voltar_consulta.on_click(voltar_para_tela_inicial)

#  Criando a Tela Inicial
def tela_inicial():
    botao_login = pn.widgets.Button(name="Login", button_type="primary")
    botao_cadastro = pn.widgets.Button(name="Cadastrar", button_type="success")

    def abrir_tela_cadastro(event):
        main_area.objects = [tela_cadastro]  

    def abrir_tela_login(event):
        main_area.objects = [tela_login]  

    botao_cadastro.on_click(abrir_tela_cadastro)
    botao_login.on_click(abrir_tela_login)

    return pn.Column(
        "# Bem-vindo ao SocioEco Data!",
        "## Escolha uma opção abaixo:",
        botao_login,
        botao_cadastro
    )

#  Área principal que muda de tela
main_area = pn.Column()
main_area.objects = [tela_inicial()]

#  Layout final
layout = pn.Row(main_area)
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'9477581e-d792-4369-bd27-29e233194b76': {'version…